In [1]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BDH-DATA-JOB") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark版本:", spark.version)

Spark版本: 3.3.3


In [2]:
! pip install geopandas shapely


Defaulting to user installation because normal site-packages is not writeable


In [3]:
id = 1
sql = """
SELECT lon, lat
FROM (
	SELECT lon, lat
	FROM time_series
	WHERE EXISTS (
		SELECT *
		FROM job
		WHERE job.id = {id}
			AND time_series.time BETWEEN job.start_time AND job.end_time
	)
) time_series;
""".format(id=id)
points_df = spark.sql(sql)
print(len(DataFrame))
points_df.show()


NameError: name 'DataFrame' is not defined

In [ ]:
# https://ncar.github.io/PySpark4Climate/tutorials/pyspark-geo-analysis/geopandas-and-spark/#Load-Data-in-a-GeoDataFrame
# 创建GeoDataFrame

lon_list = points_df.select("lon").rdd.flatMap(lambda x: x).collect()
lat_list = points_df.select("lat").rdd.flatMap(lambda x: x).collect()
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString

# 创建几何对象
geometry = [Point(xy) for xy in zip(lon_list,lat_list)]

# 创建Geopandas的GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=geometry)

gdf.show()

# # 创建曲线
# gdf['curve'] =gdf['geometry'].apply(lambda geom: LineString(geom.coords))

# # 删除重复的曲线
# gdf = gdf.drop_duplicates(subset=['group_column', 'curve'])

# # 将GeoDataFrame的几何列转换为WKT格式的字符串
# gdf['curve_wkt'] = gdf['curve'].apply(lambda x: x.wkt)

# print(gdf['curve_wkt'])


In [ ]:
from shapely.geometry import LineString

print(f"LineString contains {len(gdf['geometry'].tolist())} points.")
linestring = LineString(gdf['geometry'].tolist())


In [ ]:
sql = """
INSERT INTO gisdb.output (job_id, geoms)
VALUES (1, ST_GeomFromText('{linestring}', 4326));
""".format(linestring=linestring)
print(sql)